# 🔁 5.2 – Patrones de Comportamiento: Command, Chain of Responsibility y State

En este notebook veremos tres patrones orientados al **control del flujo y comportamiento dinámico**:

- **Command:** encapsula acciones en objetos.
- **Chain of Responsibility:** pasa peticiones a través de una cadena de manejadores.
- **State:** permite cambiar el comportamiento de un objeto según su estado interno.

---
## 🎯 Objetivos
- Comprender cómo estos patrones reducen el acoplamiento entre emisores y receptores.
- Implementarlos en TypeScript con ejemplos prácticos.
- Aplicarlos en flujos de negocio, interfaces y máquinas de estados.

In [ ]:
console.log('✅ Notebook 5.2 – Command, Chain of Responsibility y State listo para usar.');

---
## 1️⃣ Patrón Command

El patrón **Command** encapsula una petición (acción) como un objeto, lo que permite **deshacer, registrar o encolar comandos fácilmente**.

👉 Se usa en sistemas de **undo/redo**, **colas de tareas**, o **menús de acciones** en UIs.

In [ ]:
interface Command {
  execute(): void;
}

class Light {
  turnOn() { console.log('💡 Luz encendida'); }
  turnOff() { console.log('💡 Luz apagada'); }
}

class TurnOnCommand implements Command {
  constructor(private light: Light) {}
  execute() { this.light.turnOn(); }
}

class TurnOffCommand implements Command {
  constructor(private light: Light) {}
  execute() { this.light.turnOff(); }
}

class RemoteControl {
  private history: Command[] = [];
  press(command: Command) {
    command.execute();
    this.history.push(command);
  }
}

const light = new Light();
const remote = new RemoteControl();
remote.press(new TurnOnCommand(light));
remote.press(new TurnOffCommand(light));

✅ El **Command** separa el emisor (`RemoteControl`) del receptor (`Light`).

💡 Ideal para registrar acciones, implementar `undo`, o enviar comandos a través de red.

---
## 2️⃣ Patrón Chain of Responsibility

El patrón **Chain of Responsibility** pasa una solicitud a través de una **cadena de manejadores** hasta que uno la procesa.

👉 Permite evitar estructuras `if/else` largas y sustituirlas por **manejadores encadenados**.

In [ ]:
abstract class Handler {
  protected next?: Handler;
  setNext(handler: Handler): Handler {
    this.next = handler;
    return handler;
  }
  handle(request: string): void {
    if (this.next) this.next.handle(request);
  }
}

class AuthHandler extends Handler {
  handle(request: string) {
    if (request === 'auth') console.log('🔐 Autenticación verificada');
    else super.handle(request);
  }
}

class LogHandler extends Handler {
  handle(request: string) {
    console.log(`�� Log: ${request}`);
    super.handle(request);
  }
}

class ErrorHandler extends Handler {
  handle(request: string) {
    if (request === 'error') console.log('❌ Error detectado');
    else super.handle(request);
  }
}

const auth = new AuthHandler();
const log = new LogHandler();
const error = new ErrorHandler();
log.setNext(auth).setNext(error);

log.handle('auth');
log.handle('error');

✅ La **cadena** procesa peticiones en orden hasta que un manejador la resuelve.

💡 Este patrón es clave en **middleware HTTP**, **validadores**, **sistemas de autorización**, y **procesadores de eventos**.

---
## 3️⃣ Patrón State

El patrón **State** permite que un objeto cambie su comportamiento cuando cambia su estado interno.

👉 Se usa en **máquinas de estados**, **procesos con etapas** o **interfaces dinámicas**.

In [ ]:
interface State {
  next(ticket: SupportTicket): void;
  name: string;
}

class SupportTicket {
  constructor(public state: State) {}
  next() { this.state.next(this); }
}

class NewState implements State {
  name = 'Nuevo';
  next(ticket: SupportTicket) {
    console.log('📝 Ticket pasa de Nuevo → En progreso');
    ticket.state = new InProgressState();
  }
}

class InProgressState implements State {
  name = 'En progreso';
  next(ticket: SupportTicket) {
    console.log('🔧 Ticket pasa de En progreso → Resuelto');
    ticket.state = new ResolvedState();
  }
}

class ResolvedState implements State {
  name = 'Resuelto';
  next(ticket: SupportTicket) {
    console.log('✅ Ticket completado');
  }
}

const ticket = new SupportTicket(new NewState());
ticket.next();
ticket.next();
ticket.next();

✅ El **State** encapsula los estados como objetos, eliminando condicionales y mejorando la extensibilidad.

💬 Muy usado en **workflows, juegos, autómatas, y procesos de aprobación.**

---
## 4️⃣ 🧩 Ejercicio – Flujo de publicación de artículo

Implementa una máquina de estados `Article` con los estados:
- `Draft → Review → Published`

Cada estado debe imprimir el cambio de transición. Al llegar a `Published`, debe mostrar `📰 Artículo publicado con éxito!`

In [ ]:
// Escribe tu código aquí...

### ✅ Solución propuesta

In [ ]:
interface ArticleState { next(article: Article): void; }

class Article {
  constructor(public state: ArticleState) {}
  next() { this.state.next(this); }
}

class Draft implements ArticleState {
  next(article: Article) {
    console.log('✏️ Borrador → Revisión');
    article.state = new Review();
  }
}

class Review implements ArticleState {
  next(article: Article) {
    console.log('🔍 Revisión → Publicado');
    article.state = new Published();
  }
}

class Published implements ArticleState {
  next(article: Article) {
    console.log('📰 Artículo publicado con éxito!');
  }
}

const article = new Article(new Draft());
article.next();
article.next();
article.next();

---
## 5️⃣ Comparativa Command vs Chain vs State

| Característica | Command | Chain of Responsibility | State |
|----------------|----------|--------------------------|--------|
| Propósito | Encapsular una acción | Pasar una petición por una cadena de manejadores | Cambiar comportamiento según estado |
| Tipo de relación | Emisor/Receptor | Flujo secuencial | Contexto/Estado |
| Ejemplo típico | Botones, menús, undo/redo | Middleware HTTP, validación | Workflows, autómatas |
| Principio SOLID | SRP, OCP | SRP, DIP | OCP, SRP |

💬 Los tres patrones aíslan el **comportamiento variable**, favoreciendo modularidad y extensibilidad.

---
## 🧠 Resumen del notebook

- **Command:** encapsula acciones como objetos.
- **Chain of Responsibility:** distribuye solicitudes entre manejadores.
- **State:** modela comportamientos dinámicos por estado.
- Juntos permiten construir sistemas robustos, configurables y mantenibles.

➡️ Próximo módulo → **6.1 Antipatrones y Refactorización.**